# Rest Codes

## TPU Stragegy

In [ ]:
# Detect hardware
try:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address) # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu) 
  # Going back and forth between TPU and host is expensive.
  # Better to run 128 batches on the TPU before reporting back.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
  print("Number of accelerators: ", strategy.num_replicas_in_sync)
except ValueError:
  print('TPU failed to initialize.')


with strategy.scope()

## Import Dataset

In [ ]:
# Download and prepare the horses or humans dataset

splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

#####

def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

#####

SIZE = 141 #@param {type:"slider", min:64, max:300, step:1}
BATCH_SIZE = 32 #@param {type:"integer"}
IMAGE_SIZE = (SIZE, SIZE)


## Rescale

In [ ]:
# Another way to rescale img

def format_image(data):        
   image = data["image"]
   image = tf.reshape(image, [-1])
   image = tf.cast(image, 'float32')
   image = image / 255.0
   return image, data["label"]

## Custom Training

### Gradient

In [ ]:
optimizer = optimizer = tf.keras.optimizers.RMSprop()
loss_object = tf.keras.losses.BinaryCrossentropy()

train_acc_metric = tf.keras.metrics.BinaryAccuracy()
val_acc_metric = tf.keras.metrics.BinaryAccuracy()

### Training

In [ ]:
def train_data_for_one_epoch():

  losses = []
  pbar = tqdm(total=len(list(enumerate(train_generator))), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')

  for step, (x_batch_train, y_batch_train) in enumerate(train_generator):

    logits, loss_value = apply_gradient(optimizer, model, x_batch_train, y_batch_train)
    losses.append(loss_value)

    train_acc_metric(y_batch_train, logits)

    pbar.set_description("Training loss for step %s: %.4f" % (int(step), float(loss_value)))
    pbar.update()

  return losses

### Validation

In [ ]:
def perform_validation():

  losses = []

  for x_val, y_val in validation_generator:

    val_logits = model(x_val)
    val_loss = loss_object(y_true=y_val, y_pred=val_logits)
    losses.append(val_loss)

    val_acc_metric(y_val, val_logits)

  return losses

### Running the Final Model with Customized Training

In [ ]:
model = My_Custom_Model()

epochs = 3
epochs_val_losses, epochs_train_losses = [], []

for epoch in range(epochs):
  print('Start of epoch %d' % (epoch,))
  
  losses_train = train_data_for_one_epoch()
  train_acc = train_acc_metric.result()

  losses_val = perform_validation()
  val_acc = val_acc_metric.result()

  losses_train_mean = np.mean(losses_train)
  losses_val_mean = np.mean(losses_val)
  epochs_val_losses.append(losses_val_mean)
  epochs_train_losses.append(losses_train_mean)

  print('\n Epoch %s: Train loss: %.4f  Validation Loss: %.4f, Train Accuracy: %.4f, Validation Accuracy %.4f' % (epoch, float(losses_train_mean), float(losses_val_mean), float(train_acc), float(val_acc)))
  
  train_acc_metric.reset_states()
  val_acc_metric.reset_states()